In [1]:
%autosave 0
import requests
import io

from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))



## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
58,ABW,Aruba,2020-05-20,101,0,3,0,945.994,0.000,28.099,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
190,AFG,Afghanistan,2020-05-20,7653,581,178,5,196.592,14.925,4.573,...,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5
250,AGO,Angola,2020-05-20,50,2,2,0,1.521,0.061,0.061,...,2.405,1.362,5819.495,NaN,276.045,3.94,NaN,NaN,26.664,NaN


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-19,France,143133.0,0.0,28604.0,0.0,0.0,0.0
0,2020-05-19,China,84138.0,0.0,4657.0,0.0,0.0,0.0
0,2020-05-19,Italy,226632.0,0.0,32161.0,0.0,0.0,0.0
0,2020-05-19,Spain,232222.0,0.0,27788.0,0.0,0.0,0.0
0,2020-05-19,United States,1529314.0,0.0,91075.0,0.0,0.0,0.0
0,2020-05-19,World,4857233.0,0.0,321961.0,0.0,0.0,0.0
0,2020-05-19,United Kingdom,250205.0,0.0,34919.0,0.0,0.0,0.0
0,2020-05-19,Germany,175612.0,0.0,8040.0,0.0,0.0,0.0
0,2020-05-19,Iran,124365.0,0.0,7115.0,0.0,0.0,0.0
0,2020-05-19,Turkey,151856.0,0.0,4210.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")

In [6]:
## concatenate all df in 1
frames = [follow_df1,follow_df2,follow_df3,follow_df4,follow_df5,follow_df6,follow_df7,follow_df8,follow_df9,follow_df10]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-20,France,143133.0,143427.0,28604.0,28022.0,-294.0,582.0
0,2020-05-20,China,84138.0,84065.0,4657.0,4638.0,73.0,19.0
0,2020-05-20,Italy,226632.0,226699.0,32161.0,32169.0,-67.0,-8.0
0,2020-05-19,Spain,232222.0,232037.0,27788.0,27778.0,185.0,10.0
0,2020-05-20,United States,1529314.0,1528568.0,91075.0,91921.0,746.0,-846.0
0,2020-05-20,World,4857233.0,4861456.0,321961.0,322483.0,-4223.0,-522.0
0,2020-05-20,United Kingdom,250205.0,248818.0,34919.0,35341.0,1387.0,-422.0
0,2020-05-20,Germany,175612.0,176007.0,8040.0,8090.0,-395.0,-50.0
0,2020-05-20,Iran,124365.0,124603.0,7115.0,7119.0,-238.0,-4.0
0,2020-05-20,Turkey,151856.0,151615.0,4210.0,4199.0,241.0,11.0


In [7]:
## dump dataset for use it in other nb
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")